In [14]:
import numpy as np

room_temp = 65
side_length = 10

brownie_pan = np.full((side_length, side_length), room_temp)

In [18]:
brownie_density =  500
brownie_specific_heat = 434
brownie_conductivity = 59

alpha = brownie_conductivity / (brownie_density * brownie_specific_heat)

def heat_transfer_over_increment(brownie_pan: np.array):
    for x in range(side_length):
        for y in range(side_length):
            # edge
            if (x == 0 and y == 0) or (x == 0 and y == (side_length - 1)) or (x == (side_length - 1) and y == 0) or (x == (side_length - 1) and y == (side_length - 1)):
                # print('corner')
                continue
            elif x == 0 or y == 0 or x == (side_length - 1) or y == (side_length - 1):
                continue
            else:
                f_naught = alpha
                surrounding_squares = brownie_pan[x + 1][y] + brownie_pan[x - 1][y] + brownie_pan[x][y + 1] + brownie_pan[x][y - 1]
                same_square = (4 - (1/f_naught)) * brownie_pan[x][y]
                brownie_pan[x][y] = (surrounding_squares - same_square) * f_naught


brownie_pan

array([[65, 65, 65, 65, 65, 65, 65, 65, 65, 65],
       [65, 65, 65, 65, 65, 65, 65, 65, 65, 65],
       [65, 65, 65, 65, 65, 65, 65, 65, 65, 65],
       [65, 65, 65, 65, 65, 65, 65, 65, 65, 65],
       [65, 65, 65, 65, 65, 65, 65, 65, 65, 65],
       [65, 65, 65, 65, 65, 65, 65, 65, 65, 65],
       [65, 65, 65, 65, 65, 65, 65, 65, 65, 65],
       [65, 65, 65, 65, 65, 65, 65, 65, 65, 65],
       [65, 65, 65, 65, 65, 65, 65, 65, 65, 65],
       [65, 65, 65, 65, 65, 65, 65, 65, 65, 65]])